In [46]:
import random
import sympy
import time
import math

In [47]:
def dichotomy_pow_mod(x, y, n):
    q = x % n
    z = x % n if y % 2 == 1 else 1
    y = y >> 1
    while y > 0:
        q = (q * q) % n
        if y % 2 == 1:
            z = (z * q) % n
        y = y >> 1
    return z

In [48]:
def factorize(n):
    fac = factorint(n)
    return fac

In [49]:
def lucas_primality_test(n, t, factorization=None):
    if n <= 3 or n % 2 == 0:
        return False
    if factorization is None:
        fac = factorize(n - 1)
    else:
        fac = factorization
    prime_divs = list(fac.keys())
    for k in range(t):
        a = random.randrange(2, n - 1)
        if math.gcd(a, n) != 1:
            continue
        if dichotomy_pow_mod(a, n - 1, n) != 1:
            continue
        is_primitive = True
        for p in prime_divs:
            if dichotomy_pow_mod(a, (n - 1) // p, n) == 1:
                is_primitive = False
                break
        if is_primitive:
            return True
    return False

In [50]:
def is_prime_miller_rabin(n, k):
    if n <= 3:
        return False
    if n <= 1 or n % 2 == 0:
        return False
    s = 0
    r = n - 1
    while r % 2 == 0:
        s += 1
        r //= 2
    for i in range(k):
        b = random.randrange(2, n - 1)
        y = dichotomy_pow_mod(b, r, n)
        if y != 1 and y != n - 1:
            j = 1
            while (j < s) and (y != n - 1):
                y = dichotomy_pow_mod(y, 2, n)
                if y == 1: 
                    return False
                j += 1
            if y != n - 1:
                return False
    return True

In [51]:
#-----------------------------Метод Гордона-------------------------------------------------------------------------------------

In [52]:
def generate_large_prime(bits, k):
    while True:
        candidate = random.getrandbits(bits)
        candidate |= (1 << (bits - 1))  # установить старший бит
        candidate |= 1                  # сделать нечётным
        if is_prime_miller_rabin(candidate, k):
            return candidate

In [53]:
def gordon_strong_prime(bits , k):
    s = generate_large_prime(bits, k)
    t = generate_large_prime(bits, k)
    #r = 2 * i * t + 1
    i = random.randint(2, 1024)
    while True:
        r = 2 * i * t + 1
        if is_prime_miller_rabin(r, k):
            break
        i += 1
    #p0 = 2 * (s^(r-2) mod r) * s - 1
    s_exp = dichotomy_pow_mod(s, r-2, r)
    p0 = 2 * s_exp * s - 1
    #p = 2 * j * r * s + p0
    j = random.randint(2, 1024)
    while True:
        p = 2 * j * r * s + p0
        if is_prime_miller_rabin(p, k) and lucas_primality_test(p,k):
            break
        j += 1
    return p

In [45]:
bits = int(input("Введите длину простого числа в битах: "))
k = int(input("Введите количество раундов Миллера-Рабина и Метода Люка: "))

start = time.time_ns()
prime = generate_large_prime(bits, k)
elapsed = time.time_ns() - start

print(f"\nСгенерированное вероятно простое число:\n{prime}")
print(f"Длина числа: {prime.bit_length()} бит")
print(f"Время генерации: {elapsed/1e6:.2f} ms ({elapsed/1e9:.4f} s)")

Введите длину простого числа в битах: 2048
Введите количество раундов Миллера-Рабина и Метода Люка: 64

Сгенерированное вероятно простое число:
26765462464528514963302802096707862087080776128216295602908530366269573727226573020077210535380248055603297534065350664128824084757461267137525538353834623538397041184451970415368478089923542233491120925110378434457662342751794149276879983200277340672805031948589356594590131820515788959600366720701797679156230366741575797363939438297374657977805661466422529280650501222257635495309751788393814922366890268717717900739528762699817173752180660441078158806922322688797819039576066913669377420567750356399758026869327306028204043197429816961485637966717222796931686243075257781920887995845433842810937594844213421655171
Длина числа: 2048 бит
Время генерации: 95304.35 ms (95.3043 s)
